In [1]:
import os
os.chdir("/global/scratch/projects/fc_mvslab/predictors/all_predictors/")

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf # tested on version 2.2.0
import glob, pickle, os, re, subprocess, csv, sys
sys.path.append(os.path.abspath('..'))
from PADDLE import paddle

In [3]:
pad_noSS = paddle.PADDLE_noSS()

Loading model split 1 of 10


2025-05-15 11:19:21.701490: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Loading model split 2 of 10
Loading model split 3 of 10
Loading model split 4 of 10
Loading model split 5 of 10
Loading model split 6 of 10
Loading model split 7 of 10
Loading model split 8 of 10
Loading model split 9 of 10
Loading model split 10 of 10


In [5]:
# Tiling input seqs
def tile_sequence(sequence, tile_length=53, step_size=1):
    return [sequence[i:i+tile_length] for i in range(0, len(sequence) - tile_length + 1, step_size)]

In [7]:
yeast_TFs = pd.read_csv("/global/scratch/projects/fc_mvslab/OpenProjects/Sanjana/consensus_AD_predictor/output/yeast_TF_seqs.csv", index_col = 0)
yeast_TFs

,id,seq
0,sp|O14467|MBF1_YEAST Multiprotein-bridging fac...,MSDWDTNTIIGSRARAGGSGPRANVARSQGQINAARRQGLVVSVDK...
1,sp|O93958|MATA2_YARLL Mating-type protein A2 O...,MENTILHIHSFQLPQTEQPYPEAMLFDRDTSDSRTVLTQKPNGLEI...
2,sp|P03069|GCN4_YEAST General control transcrip...,MSEYQPSLFALNPMGFSPLDGSKSTNENVSASTSTAKPMVGQLIFD...
3,sp|P04386|GAL4_YEAST Regulatory protein GAL4 O...,MKLLSSIEQACDICRLKKLKCSKEKPKCAKCLKNNWECRYSPKTKR...
4,sp|P04387|GAL80_YEAST Galactose/lactose metabo...,MDYNKRSSVSTVPNAAPIRVGFVGLNAAKGWAIKTHYPAILQLSSQ...
...,...,...
242,sp|Q707Y3|MATA1_YARLL Mating-type protein A1 O...,MPSRTPTDIWRCQRLILAARKGETTCQALHEQSIEISSSLKWFEEI...
243,sp|Q707Y6|MATA1_PICAN Mating-type protein A1 O...,MQFTILNEPSLDSQRREGDLASENYVFGDIRKEGVRILEDSLRSER...
244,sp|Q708A1|MATA1_NAKDE Mating-type protein A1 O...,MNVQEIHNIREACITILSGTKHNSVLFEPCDKFDEVINSLDIDPDS...
245,sp|Q9HG12|MATA1_KLULA Mating-type protein A1 O...,MCDNDMADIQSKLSSFCEEIRALALKEGYNLEGDKSPSSKPYFMSW...


In [11]:
yeast_TFs['tiled_sequences'] = yeast_TFs['seq'].apply(tile_sequence)
yeast_TFs = yeast_TFs.explode('tiled_sequences').reset_index(drop=True)
yeast_TFs

,id,seq,tiled_sequences
0,sp|O14467|MBF1_YEAST Multiprotein-bridging fac...,MSDWDTNTIIGSRARAGGSGPRANVARSQGQINAARRQGLVVSVDK...,MSDWDTNTIIGSRARAGGSGPRANVARSQGQINAARRQGLVVSVDK...
1,sp|O14467|MBF1_YEAST Multiprotein-bridging fac...,MSDWDTNTIIGSRARAGGSGPRANVARSQGQINAARRQGLVVSVDK...,SDWDTNTIIGSRARAGGSGPRANVARSQGQINAARRQGLVVSVDKK...
2,sp|O14467|MBF1_YEAST Multiprotein-bridging fac...,MSDWDTNTIIGSRARAGGSGPRANVARSQGQINAARRQGLVVSVDK...,DWDTNTIIGSRARAGGSGPRANVARSQGQINAARRQGLVVSVDKKY...
3,sp|O14467|MBF1_YEAST Multiprotein-bridging fac...,MSDWDTNTIIGSRARAGGSGPRANVARSQGQINAARRQGLVVSVDK...,WDTNTIIGSRARAGGSGPRANVARSQGQINAARRQGLVVSVDKKYG...
4,sp|O14467|MBF1_YEAST Multiprotein-bridging fac...,MSDWDTNTIIGSRARAGGSGPRANVARSQGQINAARRQGLVVSVDK...,DTNTIIGSRARAGGSGPRANVARSQGQINAARRQGLVVSVDKKYGS...
...,...,...,...
135694,sp|Q9HG13|MATA2_KLULA Mating-type protein A2 O...,MANSTLRRTTFFKLTTTEDEDTIPKLLQPNNNSVAFPNLKRTGKAF...,TKFKYKFKKQPKTKMNSNLSKLRFKSKQPPTPPEENSNVFKKRYTS...
135695,sp|Q9HG13|MATA2_KLULA Mating-type protein A2 O...,MANSTLRRTTFFKLTTTEDEDTIPKLLQPNNNSVAFPNLKRTGKAF...,KFKYKFKKQPKTKMNSNLSKLRFKSKQPPTPPEENSNVFKKRYTSE...
135696,sp|Q9HG13|MATA2_KLULA Mating-type protein A2 O...,MANSTLRRTTFFKLTTTEDEDTIPKLLQPNNNSVAFPNLKRTGKAF...,FKYKFKKQPKTKMNSNLSKLRFKSKQPPTPPEENSNVFKKRYTSEN...
135697,sp|Q9HG13|MATA2_KLULA Mating-type protein A2 O...,MANSTLRRTTFFKLTTTEDEDTIPKLLQPNNNSVAFPNLKRTGKAF...,KYKFKKQPKTKMNSNLSKLRFKSKQPPTPPEENSNVFKKRYTSENR...


In [12]:
# Making predictions on sequences
seqs = yeast_TFs["tiled_sequences"]
preds = pad_noSS.predict(seqs)

In [13]:
# Saving predictions
yeast_TFs["paddle_noSS_pred"] = preds
yeast_TFs

,id,seq,tiled_sequences,paddle_noSS_pred
0,sp|O14467|MBF1_YEAST Multiprotein-bridging fac...,MSDWDTNTIIGSRARAGGSGPRANVARSQGQINAARRQGLVVSVDK...,MSDWDTNTIIGSRARAGGSGPRANVARSQGQINAARRQGLVVSVDK...,-0.043939
1,sp|O14467|MBF1_YEAST Multiprotein-bridging fac...,MSDWDTNTIIGSRARAGGSGPRANVARSQGQINAARRQGLVVSVDK...,SDWDTNTIIGSRARAGGSGPRANVARSQGQINAARRQGLVVSVDKK...,-0.091048
2,sp|O14467|MBF1_YEAST Multiprotein-bridging fac...,MSDWDTNTIIGSRARAGGSGPRANVARSQGQINAARRQGLVVSVDK...,DWDTNTIIGSRARAGGSGPRANVARSQGQINAARRQGLVVSVDKKY...,-0.108901
3,sp|O14467|MBF1_YEAST Multiprotein-bridging fac...,MSDWDTNTIIGSRARAGGSGPRANVARSQGQINAARRQGLVVSVDK...,WDTNTIIGSRARAGGSGPRANVARSQGQINAARRQGLVVSVDKKYG...,-0.154330
4,sp|O14467|MBF1_YEAST Multiprotein-bridging fac...,MSDWDTNTIIGSRARAGGSGPRANVARSQGQINAARRQGLVVSVDK...,DTNTIIGSRARAGGSGPRANVARSQGQINAARRQGLVVSVDKKYGS...,-0.205383
...,...,...,...,...
135694,sp|Q9HG13|MATA2_KLULA Mating-type protein A2 O...,MANSTLRRTTFFKLTTTEDEDTIPKLLQPNNNSVAFPNLKRTGKAF...,TKFKYKFKKQPKTKMNSNLSKLRFKSKQPPTPPEENSNVFKKRYTS...,-0.209127
135695,sp|Q9HG13|MATA2_KLULA Mating-type protein A2 O...,MANSTLRRTTFFKLTTTEDEDTIPKLLQPNNNSVAFPNLKRTGKAF...,KFKYKFKKQPKTKMNSNLSKLRFKSKQPPTPPEENSNVFKKRYTSE...,-0.119144
135696,sp|Q9HG13|MATA2_KLULA Mating-type protein A2 O...,MANSTLRRTTFFKLTTTEDEDTIPKLLQPNNNSVAFPNLKRTGKAF...,FKYKFKKQPKTKMNSNLSKLRFKSKQPPTPPEENSNVFKKRYTSEN...,0.053190
135697,sp|Q9HG13|MATA2_KLULA Mating-type protein A2 O...,MANSTLRRTTFFKLTTTEDEDTIPKLLQPNNNSVAFPNLKRTGKAF...,KYKFKKQPKTKMNSNLSKLRFKSKQPPTPPEENSNVFKKRYTSENR...,0.151122


In [14]:
yeast_TFs.to_csv("/global/scratch/projects/fc_mvslab/OpenProjects/Sanjana/consensus_AD_predictor/output/yeast_TF_preds/PADDLE_noSS_yeast_TFs.csv")